In [1]:
import pandas as pd
import pdb
import numpy as np
import itertools
from utils import *
from snakemake.io import expand
import yaml
import cerberus
import pyranges as pr

In [33]:
config_file = 'workflow/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [34]:
species = 'human'
pc_bed_file = expand(config['procap']['bed_format'],
                     pc_biosamp='k562',
                     pc_rep='1',
                     pc_output='bidirectional_peaks',
                     species=species)[0]
pc_bed_file_2 = expand(config['procap']['bed_format'],
                     pc_biosamp='k562',
                     pc_rep='1',
                     pc_output='unidirectional_peaks',
                     species=species)[0]
lr_bed_file = expand(config['lr']['bed'],
                     lr_biosamp='k562',
                     lr_rep='1',
                     species=species)[0]

ccre_bed_file = expand(config['ccre']['bed'],
                       species='human',
                       ccre_type='pls')[0]

In [35]:
df = pr.read_bed(pc_bed_file)
df.head()


,Chromosome,Start,End,Name,Score,Strand
0,chr1,629340,629643,.,.,.
1,chr1,629899,629978,.,.,.
2,chr1,630573,630702,.,.,.
3,chr1,634027,634244,.,.,.
4,chr1,778569,778809,.,.,.
5,chr1,827478,827702,.,.,.
6,chr1,904389,904462,.,.,.
7,chr1,904679,904849,.,.,.


In [36]:
df = pr.read_bed(pc_bed_file_2)
df.head()


,Chromosome,Start,End,Name,Score,Strand
0,chr1,115726,115733,.,.,.
1,chr1,633845,633848,.,.,.
2,chr1,633884,634026,.,.,.
3,chr1,943372,943376,.,.,.
4,chr1,944529,944539,.,.,.
5,chr1,1017579,1017584,.,.,.
6,chr1,1157495,1157520,.,.,.
7,chr1,1157655,1157728,.,.,.


In [12]:
# get lists to expand w/ 
pc_biosamps = ['k562']
pc_reps = ['1']
pc_outputs = ['bidirectional_peaks']
pc_species = ['human']

lr_biosamps = ['k562']
lr_reps = ['1']
lr_species=['human']

In [14]:
species = 'human'
pc_bed_files = expand(config['procap']['bed'],
                     pc_biosamp=pc_biosamps,
                     pc_rep=pc_reps,
                     pc_output=pc_outputs,
                     species=pc_species)
lr_bed_files = expand(config['lr']['bed'],
                     lr_biosamp=lr_biosamps,
                     lr_rep=lr_reps,
                     species=lr_species)

ccre_bed_file = expand(config['ccre']['bed'],
                       species='human',
                       ccre_type='pls')[0]

In [21]:
# lr_bed = pr.read_bed(lr_bed_file)
# pc_bed = pr.read_bed(pc_bed_file)

# loop through each procap bed file and cat them together with the relevant info
df = pd.DataFrame()
for f, b, r, o in zip(pc_bed_files,
                      pc_biosamps,
                      pc_reps,
                      pc_outputs):
    temp = pr.read_bed(f, as_df=True)
    temp['assay'] = 'procap'
    temp['biosample'] = b
    temp['rep'] = r
    temp['output'] = o
    df = pd.concat([df, temp], axis=0)
    
# do the same with the lr bed files
for f, b, r in zip(lr_bed_files,
                                 lr_biosamps,
                                 lr_reps):
    temp = pr.read_bed(f, as_df=True)
    temp['assay'] = 'lr'
    temp['biosample'] = b
    temp['rep'] = r
    df = pd.concat([df, temp], axis=0)

In [20]:
df

,Chromosome,Start,End,Name,Score,Strand,assay,biosample,rep,output
0,chr1,629340,629643,Relaxed,629639,629341,procap,k562,1,bidirectional_peaks
1,chr1,629899,629978,Stringent(qval),629909,629955,procap,k562,1,bidirectional_peaks
2,chr1,630573,630702,Relaxed,630681,630574,procap,k562,1,bidirectional_peaks
3,chr1,634027,634244,Relaxed,634243,634032,procap,k562,1,bidirectional_peaks
4,chr1,778569,778809,Stringent(qval),778808,778612,procap,k562,1,bidirectional_peaks
...,...,...,...,...,...,...,...,...,...,...
18352,chrX,155216874,155216925,Relaxed,155216924,155216883,procap,k562,1,bidirectional_peaks
18353,chrX,155334610,155334745,Relaxed,155334744,155334613,procap,k562,1,bidirectional_peaks
18354,chrX,155612927,155613058,Stringent(qval),155613031,155612933,procap,k562,1,bidirectional_peaks
18355,chrX,155881181,155881419,Stringent(pval),155881357,155881182,procap,k562,1,bidirectional_peaks


In [9]:
ccre_bed = pr.read_bed(ccre_bed_file)